We need to compile llama-cpp-python[server] using Nvidia Compile (nvcc). Make sure you are running a CUDA Notebook !

In [1]:
!NVCC_APPEND_FLAGS='-allow-unsupported-compiler' CUDACXX=/usr/local/cuda/bin/nvcc CMAKE_ARGS="-DLLAMA_CUBLAS=on -DCMAKE_CUDA_ARCHITECTURES=all-major" FORCE_CMAKE=1 pip install llama-cpp-python[server] --no-cache-dir --force-reinstall --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.4/37.4 MB 135.9 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 168.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 175.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 162.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.9/738.9 kB 212.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 194.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 161.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 187.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 228.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 224.7 MB/s eta 0

Next, install the hugging face CLI and Torch tool so we can grab a llama model to chat to.

In [2]:
pip install 'huggingface_hub[cli,torch]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 8.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 238.0 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Login to Hugging Face, you will need your API_KEY set as an environment variable. 

In [2]:
from huggingface_hub import login, logout
import os
login(os.getenv("HF_API_KEY"))

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /opt/app-root/src/.cache/huggingface/token
Login successful


Download a llama-3 model. This has a few quantized, finetuned
versions - https://huggingface.co/lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF

In [ ]:
!huggingface-cli download lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF --local-dir . --local-dir-use-symlinks False

Load the model making sure to specify to use the GPU with n_gpu_layers.

In [1]:
from llama_cpp import Llama
llm = Llama(model_path="Meta-Llama-3-8B-Instruct-Q8_0.gguf", n_gpu_layers=-1)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from Meta-Llama-3-8B-Instruct-Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Meta-Llama-3-8B-Instruct-imatrix
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:              llama.atte

This is where we specify the completions.

In [5]:
output = llm(
  "<PROMPT>", # Prompt
  max_tokens=512,  # Generate up to 512 tokens
  stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
  echo=True        # Whether to echo the prompt
)


llama_print_timings:        load time =     354.76 ms
llama_print_timings:      sample time =      94.05 ms /   180 runs   (    0.52 ms per token,  1913.98 tokens per second)
llama_print_timings: prompt eval time =      36.94 ms /     5 tokens (    7.39 ms per token,   135.35 tokens per second)
llama_print_timings:        eval time =    6300.40 ms /   179 runs   (   35.20 ms per token,    28.41 tokens per second)
llama_print_timings:       total time =    7727.27 ms /   184 tokens


In [7]:
llm.create_chat_completion(
    messages = [
        {"role": "system", "content": "You are a story writing assistant with a pirate style."},
        {
            "role": "user",
            "content": "Write a story about llamas."
        }
    ]
)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     354.76 ms
llama_print_timings:      sample time =     263.81 ms /   477 runs   (    0.55 ms per token,  1808.13 tokens per second)
llama_print_timings: prompt eval time =     137.74 ms /    23 tokens (    5.99 ms per token,   166.99 tokens per second)
llama_print_timings:        eval time =   16935.40 ms /   476 runs   (   35.58 ms per token,    28.11 tokens per second)
llama_print_timings:       total time =   20743.89 ms /   499 tokens


{'id': 'chatcmpl-e4108939-7725-48a9-952a-a3ceb8d6aabb',
 'object': 'chat.completion',
 'created': 1714008693,
 'model': 'Meta-Llama-3-8B-Instruct-Q8_0.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': "Arrrr, ye landlubbers! Listen close to the tale of the most majestic creatures on the high seas... er, I mean, in the Andean mountains: the llamas!\n\nIn the small village of Pachamama, nestled between the snow-capped peaks and lush valleys, a group o' llamas lived life like no other. They were the pride o' their owner, Señorita Sofia, who treated 'em like her own treasure.\n\nThe leader o' the pack was a wise and fierce llama named Kusi. He had a coat as white as the driven snow and eyes that shone like the stars on a clear night. Kusi had lived through many adventures with Sofia's family, from trekking to Machu Picchu to helping with harvests in the fertile valleys.\n\nOne day, a mischievous band o' raccoons, led by their cunning leader, Riki-Tiki, beg